In [1]:
import nest_asyncio
import asyncio
import BinanceTradeClient
import MarketDataFetcher
import BinanceBackTest
import utils

nest_asyncio.apply()

obj = BinanceBackTest.DataManager(symbols='BTCUSDT', intervals=['1m','5m', '1h'], start_date='2024-11-01 00:00:00', end_date='2024-11-10 00:00:00')
range_length_data, kline_data, indices_data, closing_sync_data = asyncio.run(
    obj.read_data_run(download_if_missing=False)
)

"""
idx를 넣으면 kline 데이터가 closing_sync_data와 매칭되어 반환되는 함수.
"""

'\nidx를 넣으면 kline 데이터가 closing_sync_data와 매칭되어 반환되는 함수.\n'

In [11]:
idx = 650
# data = obj.sync_kline_data(idx=)
data = obj.sync_kline_data(idx=idx, kline_data=kline_data, idx_mapping=indices_data, sync_data=closing_sync_data)


In [12]:
data.get('BTCUSDT').get('1h')[indices_data[650][2]]

array([1.73249640e+12, 9.77632000e+04, 9.79003000e+04, 9.69299000e+04,
       9.71893000e+04, 8.41452500e+03, 1.73250000e+12, 8.18902906e+08,
       1.88228000e+05, 3.83872600e+03, 3.73533161e+08, 0.00000000e+00])

In [10]:
closing_sync_data.get('BTCUSDT').get('1h')[650]

array([1.73249640e+12, 9.77632000e+04, 9.79003000e+04, 9.69299000e+04,
       9.71893000e+04, 8.41452500e+03, 1.73250000e+12, 8.18902906e+08,
       1.88228000e+05, 3.83872600e+03, 3.73533161e+08, 0.00000000e+00])